### N-gram language models or how to write scientific papers (4 pts)

We shall train our language model on a corpora of [ArXiv](http://arxiv.org/) articles and see if we can generate a new one!

![img](https://media.npr.org/assets/img/2013/12/10/istock-18586699-monkey-computer_brick-16e5064d3378a14e0e4c2da08857efe03c04695e-s800-c85.jpg)

_data by neelshah18 from [here](https://www.kaggle.com/neelshah18/arxivdataset/)_

_Disclaimer: this has nothing to do with actual science. But it's fun, so who cares?!_

In [151]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [152]:
# Alternative manual download link: https://yadi.sk/d/_nGyU2IajjR9-w
# !wget "https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1" -O arxivData.json.tar.gz
# !tar -xvzf arxivData.json.tar.gz
data = pd.read_json("./arxivData.json")
data.sample(n=5)

,author,day,id,link,month,summary,tag,title,year
11287,"[{'name': 'Tianpei Xie'}, {'name': 'Nasser M. ...",16,1507.04540v3,"[{'rel': 'related', 'href': 'http://dx.doi.org...",7,"In this paper, we propose a general framework ...","[{'term': 'cs.LG', 'scheme': 'http://arxiv.org...",Learning to classify with possible sensor fail...,2015
34505,[{'name': 'Francesco Fumarola'}],15,1611.04842v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",11,A topological argument is presented concering ...,"[{'term': 'q-bio.NC', 'scheme': 'http://arxiv....",The Role of Word Length in Semantic Topology,2016
37747,"[{'name': 'Caelan Reed Garrett'}, {'name': 'To...",23,1802.08705v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",2,Many planning applications involve complex rel...,"[{'term': 'cs.AI', 'scheme': 'http://arxiv.org...",STRIPStream: Integrating Symbolic Planners and...,2018
27672,"[{'name': 'Kyle Genova'}, {'name': 'Manolis Sa...",7,1704.02393v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",4,"The use of rendered images, whether from compl...","[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Learning Where to Look: Data-Driven Viewpoint ...,2017
33887,"[{'name': 'Tao Liu'}, {'name': 'Lei Jiang'}, {...",14,1803.05109v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",3,One of the most exciting advancements in AI ov...,"[{'term': 'cs.NE', 'scheme': 'http://arxiv.org...",PT-Spike: A Precise-Time-Dependent Single Spik...,2018


In [153]:
# assemble lines: concatenate title and description
lines = data.apply(lambda row: row['title'] + ' ; ' + row['summary'].replace("\n", ' '), axis=1).tolist()

sorted(lines, key=len)[:3]

['Differential Contrastive Divergence ; This paper has been retracted.',
 'What Does Artificial Life Tell Us About Death? ; Short philosophical essay',
 'P=NP ; We claim to resolve the P=?NP problem via a formal argument for P=NP.']

### Tokenization

You know the dril. The data is messy. Go clean the data. Use WordPunctTokenizer or something.


In [154]:
# Task: convert lines (in-place) into strings of space-separated tokens. Import & use WordPunctTokenizer

# <YOUR CODE>
# lines = <...>

from nltk.tokenize import WordPunctTokenizer
tk = WordPunctTokenizer()
for i in range(len(lines)):
    tokens = tk.tokenize(lines[i])
    tokens = [token.lower() for token in tokens]
    lines[i] = " ".join(tokens)



In [155]:
assert sorted(lines, key=len)[0] == \
    'differential contrastive divergence ; this paper has been retracted .'
assert sorted(lines, key=len)[2] == \
    'p = np ; we claim to resolve the p =? np problem via a formal argument for p = np .'

### N-Gram Language Model (1point)

A language model is a probabilistic model that estimates text probability: the joint probability of all tokens $w_t$ in text $X$: $P(X) = P(w_1, \dots, w_T)$.

It can do so by following the chain rule:
$$P(w_1, \dots, w_T) = P(w_1)P(w_2 \mid w_1)\dots P(w_T \mid w_1, \dots, w_{T-1}).$$

The problem with such approach is that the final term $P(w_T \mid w_1, \dots, w_{T-1})$ depends on $n-1$ previous words. This probability is impractical to estimate for long texts, e.g. $T = 1000$.

One popular approximation is to assume that next word only depends on a finite amount of previous words:

$$P(w_t \mid w_1, \dots, w_{t - 1}) = P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1})$$

Such model is called __n-gram language model__ where n is a parameter. For example, in 3-gram language model, each word only depends on 2 previous words.

$$
    P(w_1, \dots, w_n) = \prod_t P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1}).
$$

You can also sometimes see such approximation under the name of _n-th order markov assumption_.

The first stage to building such a model is counting all word occurences given N-1 previous words

In [156]:
from tqdm import tqdm
from collections import defaultdict, Counter

# special tokens:
# - `UNK` represents absent tokens,
# - `EOS` is a special token after the end of sequence

UNK, EOS = "_UNK_", "_EOS_"

def count_ngrams(lines, n):
    """
    Count how many times each word occured after (n - 1) previous words
    :param lines: an iterable of strings with space-separated tokens
    :returns: a dictionary { tuple(prefix_tokens): {next_token_1: count_1, next_token_2: count_2}}

    When building counts, please consider the following two edge cases:
    - if prefix is shorter than (n - 1) tokens, it should be padded with UNK. For n=3,
      empty prefix: "" -> (UNK, UNK)
      short prefix: "the" -> (UNK, the)
      long prefix: "the new approach" -> (new, approach)
    - you should add a special token, EOS, at the end of each sequence
      "... with deep neural networks ." -> (..., with, deep, neural, networks, ., EOS)
      count the probability of this token just like all others.
    """
    counts = defaultdict(Counter)
    # counts[(word1, word2)][word3] = how many times word3 occured after (word1, word2)

    # <YOUR CODE>
    prefix_len = n - 1
    for line in lines:
        tokens = line.split() + [EOS]
        for i in range(len(tokens)):
            next_token = tokens[i]
            prefix_slice = tokens[max(0, i - prefix_len) : i]
            if len(prefix_slice) < prefix_len:
                padding_needed = prefix_len - len(prefix_slice)
                prefix = tuple([UNK] * padding_needed + prefix_slice)
            else:
                prefix = tuple(prefix_slice)
            counts[prefix][next_token] += 1
            
    return counts


In [157]:
# let's test it
dummy_lines = sorted(lines, key=len)[:100]
dummy_counts = count_ngrams(dummy_lines, n=3)
assert set(map(len, dummy_counts.keys())) == {2}, "please only count {n-1}-grams"
assert len(dummy_counts[('_UNK_', '_UNK_')]) == 78
assert dummy_counts['_UNK_', 'a']['note'] == 3
assert dummy_counts['p', '=']['np'] == 2
assert dummy_counts['author', '.']['_EOS_'] == 1

Once we can count N-grams, we can build a probabilistic language model.
The simplest way to compute probabilities is in proporiton to counts:

$$ P(w_t | prefix) = { Count(prefix, w_t) \over \sum_{\hat w} Count(prefix, \hat w) } $$

In [158]:
class NGramLanguageModel:
    def __init__(self, lines, n):
        """
        Train a simple count-based language model:
        compute probabilities P(w_t | prefix) given ngram counts

        :param n: computes probability of next token given (n - 1) previous words
        :param lines: an iterable of strings with space-separated tokens
        """
        assert n >= 1
        self.n = n

        counts = count_ngrams(lines, self.n)

        # compute token proabilities given counts
        self.probs = defaultdict(Counter)
        # probs[(word1, word2)][word3] = P(word3 | word1, word2)
        for prefix in counts:
            token_counts = counts[prefix]
            total_count = sum(token_counts.values())
            for token, count in token_counts.items():
                probability = count / total_count
                self.probs[prefix][token] = probability

    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary {token : it's probability} for all tokens with positive probabilities
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]

    def get_next_token_prob(self, prefix, next_token):
        """
        :param prefix: string with space-separated prefix tokens
        :param next_token: the next token to predict probability for
        :returns: P(next_token|prefix) a single number, 0 <= P <= 1
        """
        return self.get_possible_next_tokens(prefix).get(next_token, 0)

Let's test it!

In [159]:
dummy_lm = NGramLanguageModel(dummy_lines, n=3)

p_initial = dummy_lm.get_possible_next_tokens('') # '' -> ['_UNK_', '_UNK_']
assert np.allclose(p_initial['learning'], 0.02)
assert np.allclose(p_initial['a'], 0.13)
assert np.allclose(p_initial.get('meow', 0), 0)
assert np.allclose(sum(p_initial.values()), 1)

p_a = dummy_lm.get_possible_next_tokens('a') # '' -> ['_UNK_', 'a']
assert np.allclose(p_a['machine'], 0.15384615)
assert np.allclose(p_a['note'], 0.23076923)
assert np.allclose(p_a.get('the', 0), 0)
assert np.allclose(sum(p_a.values()), 1)

assert np.allclose(dummy_lm.get_possible_next_tokens('a note')['on'], 1)
assert dummy_lm.get_possible_next_tokens('a machine') == \
    dummy_lm.get_possible_next_tokens("there have always been ghosts in a machine"), \
    "your 3-gram model should only depend on 2 previous words"

Now that you've got a working n-gram language model, let's see what sequences it can generate. But first, let's train it on the whole dataset.

In [160]:
lm = NGramLanguageModel(lines, n=3)

The process of generating sequences is... well, it's sequential. You maintain a list of tokens and iteratively add next token by sampling with probabilities.

$ X = [] $

__forever:__
* $w_{next} \sim P(w_{next} | X)$
* $X = concat(X, w_{next})$


Instead of sampling with probabilities, one can also try always taking most likely token, sampling among top-K most likely tokens or sampling with temperature. In the latter case (temperature), one samples from

$$w_{next} \sim {P(w_{next} | X) ^ {1 / \tau} \over \sum_{\hat w} P(\hat w | X) ^ {1 / \tau}}$$

Where $\tau > 0$ is model temperature. If $\tau << 1$, more likely tokens will be sampled with even higher probability while less likely tokens will vanish.

In [161]:
def get_next_token(lm, prefix, temperature=1.0):
    """
    return next token after prefix;
    :param temperature: samples proportionally to lm probabilities ^ (1 / temperature)
        if temperature == 0, always takes most likely token. Break ties arbitrarily.
    """
    # <YOUR CODE>
    tokens = lm.get_possible_next_tokens(prefix)
    words = list(tokens.keys())
    probs = list(tokens.values())
    if temperature == 0:
        idx = probs.index(max(probs))
        return words[idx]

    temp_probs = [prob ** (1 / temperature) for prob in probs]
    sum_scaled = sum(temp_probs)
    final_probs = [temp_prob / sum_scaled for temp_prob in temp_probs]
    sampled_token = np.random.choice(words, size=1, p=final_probs)[0]
    return sampled_token
    

In [162]:
from collections import Counter
test_freqs = Counter([get_next_token(lm, 'there have') for _ in range(10000)])
assert 250 < test_freqs['not'] < 450
assert 8500 < test_freqs['been'] < 9500
assert 1 < test_freqs['lately'] < 200

test_freqs = Counter([get_next_token(lm, 'deep', temperature=1.0) for _ in range(10000)])
assert 1500 < test_freqs['learning'] < 3000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.5) for _ in range(10000)])
assert 8000 < test_freqs['learning'] < 9000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.0) for _ in range(10000)])
assert test_freqs['learning'] == 10000

print("Looks nice!")

Looks nice!


Let's have fun with this model

In [163]:
prefix = 'artificial' # <- your ideas :)

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break

print(prefix)

artificial neoteny in evolutionary algorithms to characterize each article . this paper , we use the proposed support switching procedure the optimization problem . we also claim that taking into account the communication cost , which prevents it from capturing light field representation via equation constraint . by sharing learned structure across learned tasks , hand - written digits benchmark application , named sdi , for classification of hyperspectral data cubes were captured with time variant of thompson sampling , but not physically realistic observation model and present an improvement in accuracy at a distribution for choosing the regularization facilitates exploiting


In [164]:
prefix = 'bridging the' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break

print(prefix)

bridging the gap between the two - dimensional and complex data sets . we show that the proposed method of multipliers ( sadmm ), which is used to identify the contribution of this approach can be used to classify the input image . this paper , we propose a novel method to improve the performance of the main contribution of this paper , we propose a novel method for the first part of the most important tasks in computer vision . in this paper , we propose a method to learn the optimal rate , and it is possible to obtain a


__More in the homework:__ nucleus sampling, top-k sampling, beam search(not for the faint of heart).

### Evaluating language models: perplexity (1point)

Perplexity is a measure of how well your model approximates the true probability distribution behind the data. __Smaller perplexity = better model__.

To compute perplexity on one sentence, use:
$$
    {\mathbb{P}}(w_1 \dots w_N) = P(w_1, \dots, w_N)^{-\frac1N} = \left( \prod_t P(w_t \mid w_{t - n}, \dots, w_{t - 1})\right)^{-\frac1N},
$$


On the corpora level, perplexity is a product of probabilities of all tokens in all sentences to the power of $1/N$, where $N$ is __total length (in tokens) of all sentences__ in corpora.

This number can quickly get too small for float32/float64 precision, so we recommend you to first compute log-perplexity (from log-probabilities) and then take the exponent.

In [165]:
import math
import numpy as np # Used for np.log

def perplexity(lm, lines, min_logprob=np.log(10 ** -50.)):
    """
    :param lines: a list of strings with space-separated tokens
    :param min_logprob: if log(P(w | ...)) is smaller than min_logprop, set it equal to min_logrob
    :returns: corpora-level perplexity - a single scalar number from the formula above

    Note: do not forget to compute P(w_first | empty) and P(eos | full_sequence)

    PLEASE USE lm.get_next_token_prob and NOT lm.get_possible_next_tokens
    """
    
    log_prob_sum = 0.0
    total_tokens = 0
    prefix_len = lm.n - 1
    for line in lines:
        tokens = line.split() + [EOS]
        for i in range(len(tokens)):
            next_token = tokens[i]
            prefix_slice = tokens[max(0, i - prefix_len) : i]
            if len(prefix_slice) < prefix_len:
                padding_needed = prefix_len - len(prefix_slice)
                prefix = [UNK] * padding_needed + prefix_slice
            else:
                prefix = prefix_slice
            prefix = ' '.join(prefix)
            prob = lm.get_next_token_prob(prefix, next_token)
            if prob == 0:
                log_prob = min_logprob
            else:
                log_prob = math.log(prob)
                if log_prob < min_logprob:
                    log_prob = min_logprob
            log_prob_sum += log_prob
            total_tokens += 1
    exp = - (log_prob_sum / total_tokens)
    return math.e ** exp

In [166]:
lm1 = NGramLanguageModel(dummy_lines, n=1)
lm3 = NGramLanguageModel(dummy_lines, n=3)
lm10 = NGramLanguageModel(dummy_lines, n=10)

ppx1 = perplexity(lm1, dummy_lines)
ppx3 = perplexity(lm3, dummy_lines)
ppx10 = perplexity(lm10, dummy_lines)
ppx_missing = perplexity(lm3, ['the jabberwock , with eyes of flame , '])  # thanks, L. Carrol

print("Perplexities: ppx1=%.3f ppx3=%.3f ppx10=%.3f" % (ppx1, ppx3, ppx10))

assert all(0 < ppx < 500 for ppx in (ppx1, ppx3, ppx10)), "perplexity should be non-negative and reasonably small"
assert ppx1 > ppx3 > ppx10, "higher N models should overfit and "
assert np.isfinite(ppx_missing) and ppx_missing > 10 ** 6, "missing words should have large but finite perplexity. " \
    " Make sure you use min_logprob right"
assert np.allclose([ppx1, ppx3, ppx10], (318.2132342216302, 1.5199996213739575, 1.1838145037901249))

Perplexities: ppx1=318.213 ppx3=1.520 ppx10=1.184


Now let's measure the actual perplexity: we'll split the data into train and test and score model on test data only.

In [167]:
from sklearn.model_selection import train_test_split
train_lines, test_lines = train_test_split(lines, test_size=0.25, random_state=42)

for n in (1, 2, 3):
    lm = NGramLanguageModel(n=n, lines=train_lines)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))


N = 1, Perplexity = 1832.23136
N = 2, Perplexity = 85653987.28774
N = 3, Perplexity = 61999196259043170582528.00000


In [168]:
# whoops, it just blew up :)

### LM Smoothing

The problem with our simple language model is that whenever it encounters an n-gram it has never seen before, it assigns it with the probabilitiy of 0. Every time this happens, perplexity explodes.

To battle this issue, there's a technique called __smoothing__. The core idea is to modify counts in a way that prevents probabilities from getting too low. The simplest algorithm here is Additive smoothing (aka [Lapace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)):

$$ P(w_t | prefix) = { Count(prefix, w_t) + \delta \over \sum_{\hat w} (Count(prefix, \hat w) + \delta) } $$

If counts for a given prefix are low, additive smoothing will adjust probabilities to a more uniform distribution. Not that the summation in the denominator goes over _all words in the vocabulary_.

Here's an example code we've implemented for you:

In [169]:
class LaplaceLanguageModel(NGramLanguageModel):
    """ this code is an example, no need to change anything """
    def __init__(self, lines, n, delta=1.0):
        self.n = n
        counts = count_ngrams(lines, self.n)
        self.vocab = set(token for token_counts in counts.values() for token in token_counts)
        self.probs = defaultdict(Counter)

        for prefix in counts:
            token_counts = counts[prefix]
            total_count = sum(token_counts.values()) + delta * len(self.vocab)
            self.probs[prefix] = {token: (token_counts[token] + delta) / total_count
                                          for token in token_counts}
    def get_possible_next_tokens(self, prefix):
        token_probs = super().get_possible_next_tokens(prefix)
        missing_prob_total = 1.0 - sum(token_probs.values())
        missing_prob = missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        return {token: token_probs.get(token, missing_prob) for token in self.vocab}

    def get_next_token_prob(self, prefix, next_token):
        token_probs = super().get_possible_next_tokens(prefix)
        if next_token in token_probs:
            return token_probs[next_token]
        else:
            missing_prob_total = 1.0 - sum(token_probs.values())
            missing_prob_total = max(0, missing_prob_total) # prevent rounding errors
            return missing_prob_total / max(1, len(self.vocab) - len(token_probs))


**Disclaimer**: the implementation above assumes all words unknown within a given context to be equally likely, *as well as the words outside of vocabulary*. Therefore, its' perplexity will be lower than it should when encountering such words. Therefore, comparing it with a model with fewer unknown words will not be fair. When implementing your own smoothing, you may handle this by adding a virtual `UNK` token of non-zero probability. Technically, this will result in a model where probabilities do not add up to $1$, but it is close enough for a practice excercise.

In [170]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = LaplaceLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

In [171]:
for n in (1, 2, 3):
    lm = LaplaceLanguageModel(train_lines, n=n, delta=0.1)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

N = 1, Perplexity = 1832.66878
N = 2, Perplexity = 470.48021
N = 3, Perplexity = 3679.44765


In [172]:
# optional: try to sample tokens from such a model

### Kneser-Ney smoothing (2 points)

Additive smoothing is simple, reasonably good but definitely not a State of The Art algorithm.


Your final task in this notebook is to implement [Kneser-Ney](https://en.wikipedia.org/wiki/Kneser%E2%80%93Ney_smoothing) smoothing.

It can be computed recurrently, for n>1:

$$P_{kn}(w_t | prefix_{n-1}) = { \max(0, Count(prefix_{n-1}, w_t) - \delta) \over \sum_{\hat w} Count(prefix_{n-1}, \hat w)} + \lambda_{prefix_{n-1}} \cdot P_{kn}(w_t | prefix_{n-2})$$

where
- $prefix_{n-1}$ is a tuple of {n-1} previous tokens
- $lambda_{prefix_{n-1}}$ is a normalization constant chosen so that probabilities add up to 1
- Unigram $P_{kn}(w_t | prefix_{n-2})$ corresponds to Kneser Ney smoothing for {N-1}-gram language model.
- Unigram $P_{kn}(w_t)$ is a special case: how likely it is to see x_t in an unfamiliar context

See lecture slides or wiki for more detailed formulae.

__Your task__ is to
- implement `KneserNeyLanguageModel` class,
- test it on 1-3 gram language models
- find optimal (within reason) smoothing delta for 3-gram language model with Kneser-Ney smoothing

In [173]:
class KneserNeyLanguageModel(NGramLanguageModel):
    """Optimized Kneser-Ney language model implementation."""
    
    def __init__(self, lines, n, delta=0.75):
        self.n = n
        self.delta = delta
        self.counts = defaultdict(Counter)
        self.continuation_counts = defaultdict(set)
        self.vocab = set()
        
        # Cache for probabilities
        self.prob_cache = {}
        self.continuation_probs = {}
        
        # Build n-gram counts and vocabulary
        self._build_ngram_counts(lines)
        self._build_vocabulary(lines)
        
        # Build continuation counts
        self._build_continuation_counts()
        
        # Precompute continuation probabilities
        self._precompute_continuation_probs()
        
        # Precompute probabilities for all contexts
        self._precompute_all_probabilities()
    
    def _build_vocabulary(self, lines):
        """Build vocabulary from all tokens in the lines."""
        for line in lines:
            tokens = line.split()
            for token in tokens:
                self.vocab.add(token)
        self.vocab.add('_EOS_')
        self.vocab.add('_UNK_')
    
    def _build_ngram_counts(self, lines):
        """Build n-gram counts from input lines."""
        for line in lines:
            tokens = line.split() + ['_EOS_']
            for i in range(len(tokens)):
                for order in range(1, self.n + 1):
                    if i + order <= len(tokens):
                        ngram = tuple(tokens[i:i + order])
                        context = ngram[:-1]
                        word = ngram[-1]
                        self.counts[context][word] += 1
    
    def _build_continuation_counts(self):
        """Build continuation counts for all n-gram orders."""
        for context, word_counts in self.counts.items():
            for word in word_counts:
                self.continuation_counts[word].add(context)
    
    def _precompute_continuation_probs(self):
        """Precompute unigram continuation probabilities."""
        total_continuation_count = sum(len(predecessors) for predecessors in self.continuation_counts.values())
        
        for word in self.vocab:
            continuation_count = len(self.continuation_counts.get(word, set()))
            self.continuation_probs[word] = continuation_count / total_continuation_count if total_continuation_count > 0 else 1.0 / len(self.vocab)
    
    def _calculate_lambda(self, context):
        """Calculate lambda for a given context."""
        if context not in self.counts:
            return 1.0
        
        num_following_words = len(self.counts[context])
        total_context_count = sum(self.counts[context].values())
        
        if total_context_count == 0:
            return 1.0
        
        return (self.delta * num_following_words) / total_context_count
    
    def _p_kn_cached(self, word, context):
        """Cached recursive Kneser-Ney probability calculation."""
        cache_key = (word, context)
        if cache_key in self.prob_cache:
            return self.prob_cache[cache_key]
        
        order = len(context) + 1
        
        if order == 1:
            # Base case: use precomputed continuation probability
            result = self.continuation_probs.get(word, 1.0 / len(self.vocab))
        else:
            if context in self.counts:
                total_context_count = sum(self.counts[context].values())
                word_count = self.counts[context].get(word, 0)
                
                # Discounted probability
                discounted_prob = max(word_count - self.delta, 0) / total_context_count
                
                # Lambda and backoff
                lambda_val = self._calculate_lambda(context)
                backoff_context = context[1:] if len(context) > 1 else ()
                backoff_prob = self._p_kn_cached(word, backoff_context)
                
                result = discounted_prob + lambda_val * backoff_prob
            else:
                # Context not seen, back off
                backoff_context = context[1:] if len(context) > 1 else ()
                result = self._p_kn_cached(word, backoff_context)
        
        self.prob_cache[cache_key] = result
        return result
    
    def _precompute_all_probabilities(self):
        """Precompute probabilities for all contexts and words."""
        # Get all contexts that appear in the data
        all_contexts = set(self.counts.keys())
        # Add lower-order contexts for backoff
        for context in list(all_contexts):
            for i in range(1, len(context)):
                all_contexts.add(context[i:])
        
        # Precompute for each context and each word in vocabulary
        total_combinations = len(all_contexts) * len(self.vocab)
        processed = 0
        
        for context in all_contexts:
            for word in self.vocab:
                self._p_kn_cached(word, context)
                processed += 1
    
    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary where each key is a next token and each value is P(next_token|prefix)
        """
        prefix_tokens = prefix.split() if prefix else []
        
        # Convert to context tuple
        if len(prefix_tokens) >= self.n - 1:
            context = tuple(prefix_tokens[-(self.n-1):])
        else:
            context = tuple(prefix_tokens)
        
        # Use cached probabilities
        next_tokens = {}
        total_prob = 0.0
        
        for word in self.vocab:
            prob = self._p_kn_cached(word, context)
            next_tokens[word] = prob
            total_prob += prob
        
        # Normalize
        if total_prob > 0:
            for word in next_tokens:
                next_tokens[word] /= total_prob
        
        return next_tokens
    
    def get_next_token_prob(self, prefix, next_token):
        """
        :param prefix: string with space-separated prefix tokens
        :param next_token: the next token to predict probability for
        :returns: P(next_token|prefix) a single number, 0 <= P <= 1
        """
        prefix_tokens = prefix.split() if prefix else []
        
        # Convert to context tuple
        if len(prefix_tokens) >= self.n - 1:
            context = tuple(prefix_tokens[-(self.n-1):])
        else:
            context = tuple(prefix_tokens)
        
        # Get from cache
        prob = self._p_kn_cached(next_token, context)
        
        # We need to normalize - get all probabilities for this context and normalize
        total_prob = 0.0
        for word in self.vocab:
            total_prob += self._p_kn_cached(word, context)
        
        return prob / total_prob if total_prob > 0 else 0.0

In [178]:
print(len(train_lines))
print(len(test_lines))
print(len(dummy_lines))
print(len(lines))

30750
10250
100
41000


In [176]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = KneserNeyLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

In [181]:
for n in (1, 2, 3):
    lm = KneserNeyLanguageModel(train_lines[:100], n=n)
    ppx = perplexity(lm, test_lines[:100])
    print("N = %i, Perplexity = %.5f" % (n, ppx))

N = 1, Perplexity = 3287.14039
N = 2, Perplexity = 386.91637
N = 3, Perplexity = 376.96870


In [ ]:
for n in (1, 2, 3):
    lm = KneserNeyLanguageModel(train_lines, n=n)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))